# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import os
import re
import math

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = pd.read_excel('CMO_CDTractsPrecincts_220406.xlsx')
dist = dist[['Council District',
             'Tract 2020 ID\n(Use with 2020 Decennial Census)',
             'Percentage of Tract 2020 in Council District']]
dist.columns = ['council','tract','percent']
dist = dist.iloc[1:].reset_index(drop=True)
dist.tract = dist.tract.astype(str)
dist.council = np.where(dist.council == 'Outside CoSA', 0, dist.council)
dist.head()

,council,tract,percent
0,1,110100,0.999534
1,1,110300,0.499153
2,1,110500,0.000150
3,1,110600,0.002611
4,1,110700,0.953113


## Census Data

This is all the data pulled from the census. 

In [3]:
filename = 'ACSDP5Y2021.DP02-Data.csv'
filename2 = 'ACSDP5Y2021.DP02-Column-Metadata.csv'

In [4]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True).replace('N',np.nan)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp02_0001e,dp02_0001ea,dp02_0001m,dp02_0001ma,dp02_0002e,dp02_0002m,dp02_0002ma,dp02_0002ea,dp02_0003e,...,dp02_0152pma,dp02_0153pe,dp02_0153pm,dp02_0153pma,dp02_0153pea,dp02_0154pe,dp02_0154pm,dp02_0154pma,dp02_0154pea,unnamed: 1234
0,110100,1895,NaN,286,NaN,283,155,NaN,NaN,15,...,NaN,92.3,5.3,NaN,NaN,89.0,6.3,NaN,NaN,NaN
1,110300,1393,NaN,251,NaN,335,168,NaN,NaN,241,...,NaN,94.9,5.0,NaN,NaN,75.6,12.3,NaN,NaN,NaN
2,110500,921,NaN,133,NaN,128,51,NaN,NaN,25,...,NaN,79.9,5.7,NaN,NaN,73.5,7.3,NaN,NaN,NaN
3,110600,1551,NaN,269,NaN,126,77,NaN,NaN,7,...,NaN,69.3,10.1,NaN,NaN,57.8,9.9,NaN,NaN,NaN
4,110700,513,NaN,104,NaN,134,56,NaN,NaN,84,...,NaN,86.7,5.9,NaN,NaN,64.1,14.6,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [5]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['HOUSEHOLDS BY TYPE', 'RELATIONSHIP', 'MARITAL STATUS',
       'FERTILITY', 'GRANDPARENTS', 'SCHOOL ENROLLMENT',
       'EDUCATIONAL ATTAINMENT', 'VETERAN STATUS',
       'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION',
       'RESIDENCE 1 YEAR AGO', 'PLACE OF BIRTH',
       'U.S. CITIZENSHIP STATUS', 'YEAR OF ENTRY',
       'WORLD REGION OF BIRTH OF FOREIGN BORN', 'LANGUAGE SPOKEN AT HOME',
       'ANCESTRY', 'COMPUTERS AND INTERNET USE'], dtype=object)

## Functions to combine everything

In [6]:
def cal_counts(label):
    #pull out estimate and moa
    name_est = label + 'e'
    name_moa = label + 'm'

    #find the corresponding title to census label
    string = labels [labels.column_name == name_est].label.iloc[0]
#     label_title = re.findall('!!([\w\s]+)$',string)[0]
    if not string.split('!!')[3:]:
        label_title = string.split('!!')[-1]
    else:
        label_title = '!!'.join(string.split('!!')[3:])

    #isolate just one indicator
    data = dataall[['geo_id', name_est, name_moa]].copy()

    #join districts and indicator together
    df = dist.merge(data, how='inner', right_on='geo_id', left_on='tract')
    df = df.replace(np.nan, 0)

    #get count percentages for each census tract per district
    df['tru_count'] = df[name_est].astype(float) * df.percent
    df['tru_error'] = df[name_moa].astype(float) * df.percent

    #for each council district, sum up counts, and calculate new moe
    council_counts = []
    total_counts = df.tru_count.sum()

    for x in df.council.unique():
        subset = df [df.council == x]

        count = round(subset.tru_count.sum(), 2)
        count_perc = round(count / total_counts, 2)

        error = round((subset.tru_error ** 2).sum()**.5, 2)
        error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)

        council_counts.append([x, count, count_perc, error, error_perc])

    #convert to dataframe and format
    dff = pd.DataFrame(council_counts).sort_values(0)
    dff.columns = [label_title, 'counts', 'count_perc', 'moe', 'moe_perc']
    dff = dff.set_index(label_title, drop=True)
#     print(dff)

    return dff

In [7]:
def cal_district_numbers(labels_check):
    district_totals = pd.DataFrame(np.arange(11))
    totals = []

    for label in labels_check:
        dff = cal_counts(label)
        name = dff.index.name
        dff.columns = [name + ' counts', name + ' counts percent', name + ' moe', name + ' moa percent']
        dff = dff.reset_index(drop=True)

        district_totals = pd.concat([district_totals,dff],axis=1)

    district_totals = district_totals.drop(columns=0)
    return district_totals.T

## Now calculate! 

### Labor force

In [8]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-13]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final1 = cal_district_numbers(labels_check)
final1

HOUSEHOLDS BY TYPE

Estimate!!HOUSEHOLDS BY TYPE!!Total households
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household!!With children of the householder under 18 years
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With children of the householder under 18 years
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!With children of the householder under 18 years
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over
Estimate!!HOUSEHOLDS BY TYPE!!Total households!!F

,0,1,2,3,4,5,6,7,8,9,10
Total households counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Total households counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total households moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Total households moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Married-couple household counts,101264.98,17017.24,18398.11,19264.13,18574.66,13350.28,25848.60,21804.37,24959.58,30746.47,26531.59
...,...,...,...,...,...,...,...,...,...,...,...
Average household size moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.01,0.01
Average family size counts,277.94,100.30,110.02,108.26,88.48,107.57,100.39,99.59,90.49,92.67,88.34
Average family size counts percent,0.22,0.08,0.09,0.09,0.07,0.09,0.08,0.08,0.07,0.07,0.07
Average family size moe,2.71,2.24,2.85,2.01,1.85,2.37,2.54,1.96,1.54,1.53,1.50


### Relationship

In [9]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

final2 = cal_district_numbers(labels_check)    
final2

RELATIONSHIP

Estimate!!RELATIONSHIP!!Population in households
Estimate!!RELATIONSHIP!!Population in households!!Householder
Estimate!!RELATIONSHIP!!Population in households!!Spouse
Estimate!!RELATIONSHIP!!Population in households!!Unmarried partner
Estimate!!RELATIONSHIP!!Population in households!!Child
Estimate!!RELATIONSHIP!!Population in households!!Other relatives
Estimate!!RELATIONSHIP!!Population in households!!Other nonrelatives


,0,1,2,3,4,5,6,7,8,9,10
Population in households counts,528219.13,121056.09,140339.39,139492.86,131816.47,121296.59,167605.56,144746.67,162447.59,154648.56,148425.08
Population in households counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.09,0.07,0.08,0.08,0.08
Population in households moe,8622.60,3744.45,4437.75,4023.51,4149.30,4154.10,5748.32,4415.24,4009.76,4719.45,3877.83
Population in households moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Householder counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Householder counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Householder moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Householder moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Spouse counts,100750.03,17025.15,18424.95,19769.13,18347.16,13592.17,25780.86,21739.03,24817.38,30502.55,26289.59
Spouse counts percent,0.32,0.05,0.06,0.06,0.06,0.04,0.08,0.07,0.08,0.10,0.08


### Marital Status

In [10]:
grouping = groups[2]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final3 = cal_district_numbers(labels_check)
final3

MARITAL STATUS

Estimate!!MARITAL STATUS!!Males 15 years and over
Estimate!!MARITAL STATUS!!Males 15 years and over!!Never married
Estimate!!MARITAL STATUS!!Males 15 years and over!!Now married, except separated
Estimate!!MARITAL STATUS!!Males 15 years and over!!Separated
Estimate!!MARITAL STATUS!!Males 15 years and over!!Widowed
Estimate!!MARITAL STATUS!!Males 15 years and over!!Divorced
Estimate!!MARITAL STATUS!!Females 15 years and over
Estimate!!MARITAL STATUS!!Females 15 years and over!!Never married
Estimate!!MARITAL STATUS!!Females 15 years and over!!Now married, except separated
Estimate!!MARITAL STATUS!!Females 15 years and over!!Separated
Estimate!!MARITAL STATUS!!Females 15 years and over!!Widowed
Estimate!!MARITAL STATUS!!Females 15 years and over!!Divorced


,0,1,2,3,4,5,6,7,8,9,10
Males 15 years and over counts,205242.25,50913.84,54488.83,54663.72,48806.15,48260.05,64077.83,57010.47,69521.11,60943.22,57662.52
Males 15 years and over counts percent,0.27,0.07,0.07,0.07,0.06,0.06,0.08,0.07,0.09,0.08,0.07
Males 15 years and over moe,4403.65,1872.36,2131.38,2002.52,1807.15,2380.11,2569.68,2116.52,2268.97,1990.62,1690.17
Males 15 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02
Never married counts,68422.69,22581.93,24351.83,22123.85,20232.01,21834.55,26249.74,23135.69,32930.16,20125.06,20447.49
Never married counts percent,0.23,0.07,0.08,0.07,0.07,0.07,0.09,0.08,0.11,0.07,0.07
Never married moe,3163.26,1387.98,1645.17,1526.80,1395.03,1625.00,1901.85,1568.69,1870.97,1347.77,1208.51
Never married moa percent,0.03,0.04,0.04,0.04,0.04,0.05,0.04,0.04,0.03,0.04,0.04
"Now married, except separated counts",112204.34,20108.86,21735.76,22830.44,21891.39,17015.69,29482.61,24870.21,28282.56,33620.26,29913.88
"Now married, except separated counts percent",0.31,0.06,0.06,0.06,0.06,0.05,0.08,0.07,0.08,0.09,0.08


### Fertility

In [11]:
grouping = groups[3]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final4 = cal_district_numbers(labels_check)
final4

FERTILITY
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)!!Per 1,000 unmarried women
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 15 to 50 years old
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 15 to 19 years old
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 20 to 34 years old
Estimate!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Per 1,000 women 35 to 50 years old


,0,1,2,3,4,5,6,7,8,9,10
Number of women 15 to 50 years old who had a birth in the past 12 months counts,7518.17,1722.05,2380.88,1836.44,1729.15,2036.56,2357.70,2127.77,2270.83,1910.05,2101.40
Number of women 15 to 50 years old who had a birth in the past 12 months counts percent,0.27,0.06,0.09,0.07,0.06,0.07,0.08,0.08,0.08,0.07,0.08
Number of women 15 to 50 years old who had a birth in the past 12 months moe,686.55,356.06,402.40,338.76,324.74,327.50,491.76,379.97,393.71,371.58,384.40
Number of women 15 to 50 years old who had a birth in the past 12 months moa percent,0.06,0.13,0.10,0.11,0.11,0.10,0.13,0.11,0.11,0.12,0.11
"Unmarried women (widowed, divorced, and never married) counts",2074.15,650.74,1134.47,795.41,815.67,1041.27,1165.71,1004.51,829.20,472.05,434.81
"Unmarried women (widowed, divorced, and never married) counts percent",0.20,0.06,0.11,0.08,0.08,0.10,0.11,0.10,0.08,0.05,0.04
"Unmarried women (widowed, divorced, and never married) moe",419.24,249.40,291.50,218.39,256.25,236.64,425.78,288.42,277.33,201.55,181.52
"Unmarried women (widowed, divorced, and never married) moa percent",0.12,0.23,0.16,0.17,0.19,0.14,0.22,0.17,0.20,0.26,0.25
"Unmarried women (widowed, divorced, and never married)!!Per 1,000 unmarried women counts",2711.89,966.24,1464.90,1317.87,866.75,1655.14,1152.15,1194.39,845.11,736.92,624.64
"Unmarried women (widowed, divorced, and never married)!!Per 1,000 unmarried women counts percent",0.20,0.07,0.11,0.10,0.06,0.12,0.09,0.09,0.06,0.05,0.05


### Grandparents

In [12]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final5 = cal_district_numbers(labels_check)
final5

GRANDPARENTS

Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Grandparents responsible for grandchildren
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years
Estimate!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years
Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years
Estimate!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren un

,0,1,2,3,4,5,6,7,8,9,10
Number of grandparents living with own grandchildren under 18 years counts,15165.64,3001.48,4599.02,5933.13,5228.28,5451.18,5708.04,4444.11,2498.42,1937.08,3072.62
Number of grandparents living with own grandchildren under 18 years counts percent,0.27,0.05,0.08,0.10,0.09,0.10,0.10,0.08,0.04,0.03,0.05
Number of grandparents living with own grandchildren under 18 years moe,1302.47,466.19,614.58,775.43,667.80,672.43,932.06,643.18,513.23,492.35,550.26
Number of grandparents living with own grandchildren under 18 years moa percent,0.05,0.09,0.08,0.08,0.08,0.07,0.10,0.09,0.12,0.15,0.11
Grandparents responsible for grandchildren counts,4416.09,921.49,1621.41,1919.18,2083.80,1682.52,1431.15,1465.98,794.99,405.77,1178.62
Grandparents responsible for grandchildren counts percent,0.25,0.05,0.09,0.11,0.12,0.09,0.08,0.08,0.04,0.02,0.07
Grandparents responsible for grandchildren moe,670.68,260.30,335.84,463.50,472.96,332.66,430.30,355.69,294.52,185.28,358.31
Grandparents responsible for grandchildren moa percent,0.09,0.17,0.13,0.15,0.14,0.12,0.18,0.15,0.23,0.28,0.18
Years responsible for grandchildren!!Less than 1 year counts,650.43,52.18,187.72,261.04,296.19,302.34,344.24,476.84,115.68,69.76,129.59
Years responsible for grandchildren!!Less than 1 year counts percent,0.23,0.02,0.07,0.09,0.10,0.10,0.12,0.17,0.04,0.02,0.04


### School Enrollment

In [13]:
grouping = groups[5]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final6 = cal_district_numbers(labels_check)
final6

SCHOOL ENROLLMENT

Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)
Estimate!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school


,0,1,2,3,4,5,6,7,8,9,10
Population 3 years and over enrolled in school counts,155352.49,29223.30,37743.55,36585.52,39106.88,32152.04,47096.17,37502.70,49523.10,39713.47,36630.78
Population 3 years and over enrolled in school counts percent,0.29,0.05,0.07,0.07,0.07,0.06,0.09,0.07,0.09,0.07,0.07
Population 3 years and over enrolled in school moe,4525.78,1709.07,2131.95,1775.15,1926.47,1964.28,2661.62,2265.56,2219.52,2320.43,1946.78
Population 3 years and over enrolled in school moa percent,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.04,0.03,0.04,0.03
"Nursery school, preschool counts",8583.19,1712.33,2444.08,2023.37,2017.22,1643.30,2490.96,1676.00,2142.27,2217.14,2391.14
"Nursery school, preschool counts percent",0.29,0.06,0.08,0.07,0.07,0.06,0.08,0.06,0.07,0.08,0.08
"Nursery school, preschool moe",932.54,321.06,515.52,344.71,386.49,350.12,804.04,333.89,355.53,482.21,420.15
"Nursery school, preschool moa percent",0.07,0.11,0.13,0.10,0.12,0.13,0.20,0.12,0.10,0.13,0.11
Kindergarten counts,7829.27,1697.74,1886.39,2219.95,2245.35,2731.52,2410.52,1835.86,2121.40,2036.99,1687.02
Kindergarten counts percent,0.27,0.06,0.07,0.08,0.08,0.10,0.08,0.06,0.07,0.07,0.06


### Educational Attainment

In [14]:
grouping = groups[6]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final7 = cal_district_numbers(labels_check)
final7

EDUCATIONAL ATTAINMENT

Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher
Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher


,0,1,2,3,4,5,6,7,8,9,10
Population 25 years and over counts,339470.70,84245.37,89685.76,90584.14,79336.06,78997.16,105213.57,98046.02,105229.63,107596.48,100442.12
Population 25 years and over counts percent,0.27,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.08,0.08,0.08
Population 25 years and over moe,5363.93,2458.02,2816.21,2574.38,2352.34,2783.11,3312.88,2678.21,2649.19,2768.51,2314.15
Population 25 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01
Less than 9th grade counts,12495.96,8943.61,8468.12,10442.06,9331.20,15000.03,6406.01,6667.88,3306.98,1887.32,3425.83
Less than 9th grade counts percent,0.14,0.10,0.10,0.12,0.11,0.17,0.07,0.08,0.04,0.02,0.04
Less than 9th grade moe,1103.05,892.42,816.04,903.11,922.63,1020.16,854.63,749.56,617.72,336.83,631.19
Less than 9th grade moa percent,0.05,0.06,0.06,0.05,0.06,0.04,0.08,0.07,0.11,0.11,0.11
"9th to 12th grade, no diploma counts",17672.58,9699.45,9715.41,13110.92,10731.65,14671.99,6014.79,7573.91,3673.13,3639.94,4192.23
"9th to 12th grade, no diploma counts percent",0.18,0.10,0.10,0.13,0.11,0.15,0.06,0.08,0.04,0.04,0.04


### Veteran status

In [15]:
grouping = groups[7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final8 = cal_district_numbers(labels_check)
final8

VETERAN STATUS

Estimate!!VETERAN STATUS!!Civilian population 18 years and over
Estimate!!VETERAN STATUS!!Civilian population 18 years and over!!Civilian veterans


,0,1,2,3,4,5,6,7,8,9,10
Civilian population 18 years and over counts,376497.03,98306.70,101632.46,103794.97,92102.26,90757.49,121094.88,112175.67,132219.87,118689.55,113149.12
Civilian population 18 years and over counts percent,0.26,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.09,0.08,0.08
Civilian population 18 years and over moe,6141.20,2774.30,3170.71,2965.41,2759.62,3110.59,3789.16,3099.68,3096.22,3135.41,2617.57
Civilian population 18 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.01
Civilian veterans counts,51541.58,6222.17,9775.34,7282.64,7245.95,3740.91,12492.56,9764.48,10478.86,12097.48,13411.03
Civilian veterans counts percent,0.36,0.04,0.07,0.05,0.05,0.03,0.09,0.07,0.07,0.08,0.09
Civilian veterans moe,2072.28,536.32,750.18,667.70,562.75,376.72,1030.37,787.35,815.22,873.52,792.04
Civilian veterans moa percent,0.02,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.05,0.04,0.04


### DISABILITY STATUS

In [16]:
grouping = groups[8]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final9 = cal_district_numbers(labels_check)
final9

DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION

Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population!!With a disability
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Under 18 years
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Under 18 years!!With a disability
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!18 to 64 years
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!18 to 64 years!!With a disability
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!65 years and over
Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!65 years and over!!With a disability


,0,1,2,3,4,5,6,7,8,9,10
Total Civilian Noninstitutionalized Population counts,522531.05,123353.54,139375.51,139651.49,131328.75,121988.46,166066.43,145755.40,164924.96,153865.12,147755.28
Total Civilian Noninstitutionalized Population counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.08,0.07,0.08,0.08,0.08
Total Civilian Noninstitutionalized Population moe,8575.67,3764.04,4442.13,4026.04,4157.12,4157.79,5755.08,4424.54,4046.24,4726.86,3844.10
Total Civilian Noninstitutionalized Population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02
With a disability counts,62784.95,17919.60,23762.66,27182.89,23864.29,25102.75,20871.52,22407.08,17758.80,16310.95,19366.51
With a disability counts percent,0.23,0.06,0.09,0.10,0.09,0.09,0.08,0.08,0.06,0.06,0.07
With a disability moe,2551.75,1075.74,1487.93,1617.61,1652.22,1426.53,1549.64,1378.17,1292.78,1206.40,1341.07
With a disability moa percent,0.02,0.04,0.04,0.04,0.04,0.03,0.05,0.04,0.04,0.04,0.04
Under 18 years counts,149749.46,26141.76,38397.56,36739.70,39597.05,34109.83,45086.55,34312.11,33333.69,35504.70,35016.58
Under 18 years counts percent,0.29,0.05,0.08,0.07,0.08,0.07,0.09,0.07,0.07,0.07,0.07


### RESIDENCE 1 YEAR AGO

In [17]:
grouping = groups[9]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final10 = cal_district_numbers(labels_check)
final10

RESIDENCE 1 YEAR AGO

Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Same house
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Same county
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Different county
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Different county!!Same state
Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house (in the U.S. or abroad)!!Different house in the U.S.!!Different county!!Different state
Estimat

,0,1,2,3,4,5,6,7,8,9,10
Population 1 year and over counts,531577.28,123804.09,141185.64,139437.43,131187.40,123775.17,165586.12,145459.26,164541.60,153780.52,147125.49
Population 1 year and over counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.08,0.07,0.08,0.08,0.07
Population 1 year and over moe,9053.93,3730.09,4427.77,3995.79,4137.45,4409.71,5613.42,4419.15,4011.25,4681.95,3824.14
Population 1 year and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02
Same house counts,450899.89,103767.31,117805.54,122623.23,115716.52,108575.36,140614.62,120985.30,118511.78,124190.69,125012.76
Same house counts percent,0.27,0.06,0.07,0.07,0.07,0.07,0.09,0.07,0.07,0.08,0.08
Same house moe,8568.02,3549.19,4294.16,3934.37,4096.94,3955.46,5340.95,4060.75,3620.35,4348.73,3677.39
Same house moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Different house (in the U.S. or abroad) counts,80677.39,20036.78,23380.10,16814.20,15470.88,15199.81,24971.50,24473.97,46029.81,29589.83,22112.73
Different house (in the U.S. or abroad) counts percent,0.25,0.06,0.07,0.05,0.05,0.05,0.08,0.08,0.14,0.09,0.07


### Place of birth

In [18]:
grouping = groups[10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final11 = cal_district_numbers(labels_check)
final11

PLACE OF BIRTH

Estimate!!PLACE OF BIRTH!!Total population
Estimate!!PLACE OF BIRTH!!Total population!!Native
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in United States
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in United States!!State of residence
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in United States!!Different state
Estimate!!PLACE OF BIRTH!!Total population!!Native!!Born in Puerto Rico, U.S. Island areas, or born abroad to American parent(s)
Estimate!!PLACE OF BIRTH!!Total population!!Foreign born


,0,1,2,3,4,5,6,7,8,9,10
Total population counts,538323.84,125014.68,143060.08,140873.10,132566.94,125056.03,167877.44,147037.45,166422.74,155155.67,149134.04
Total population counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.08,0.07,0.08,0.08,0.07
Total population moe,9093.05,3793.28,4486.09,4040.89,4164.44,4441.76,5753.03,4444.57,4049.76,4723.08,3877.11
Total population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02
Native counts,483013.99,101593.87,124192.23,123395.95,112278.66,102472.98,148448.73,127089.34,136653.67,137308.81,132946.76
Native counts percent,0.28,0.06,0.07,0.07,0.06,0.06,0.09,0.07,0.08,0.08,0.08
Native moe,8582.55,3381.72,4212.96,3696.53,3810.35,4162.03,5545.11,4009.48,3676.67,4530.44,3717.17
Native moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Native!!Born in United States counts,464740.24,99233.34,119807.78,121882.34,109685.45,101295.61,143194.64,124073.76,132500.94,132729.70,128470.20
Native!!Born in United States counts percent,0.28,0.06,0.07,0.07,0.07,0.06,0.09,0.07,0.08,0.08,0.08


### U.S. CITIZENSHIP STATUS

In [19]:
grouping = groups[11]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final12 = cal_district_numbers(labels_check)
final12

U.S. CITIZENSHIP STATUS

Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population
Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Naturalized U.S. citizen
Estimate!!U.S. CITIZENSHIP STATUS!!Foreign-born population!!Not a U.S. citizen


,0,1,2,3,4,5,6,7,8,9,10
Foreign-born population counts,55309.85,23420.81,18867.85,17477.15,20288.28,22583.05,19428.71,19948.11,29769.07,17846.86,16187.28
Foreign-born population counts percent,0.21,0.09,0.07,0.07,0.08,0.09,0.07,0.08,0.11,0.07,0.06
Foreign-born population moe,2726.58,1770.44,1514.67,1545.07,1466.02,1576.46,1723.53,1822.26,2141.51,1445.74,1484.54
Foreign-born population moa percent,0.03,0.05,0.05,0.05,0.04,0.04,0.05,0.06,0.04,0.05,0.06
Naturalized U.S. citizen counts,30774.70,6830.81,7023.32,6441.99,8672.80,6706.85,9389.93,9044.65,12330.15,9605.11,8382.68
Naturalized U.S. citizen counts percent,0.27,0.06,0.06,0.06,0.08,0.06,0.08,0.08,0.11,0.08,0.07
Naturalized U.S. citizen moe,1909.52,681.30,758.76,809.01,819.27,716.70,1009.92,1279.67,1092.83,819.52,828.85
Naturalized U.S. citizen moa percent,0.04,0.06,0.07,0.08,0.06,0.06,0.07,0.09,0.05,0.05,0.06
Not a U.S. citizen counts,24535.15,16589.99,11844.53,11035.16,11615.47,15876.20,10038.78,10903.46,17438.92,8241.75,7804.59
Not a U.S. citizen counts percent,0.17,0.11,0.08,0.08,0.08,0.11,0.07,0.07,0.12,0.06,0.05


### YEAR OF ENTRY

In [20]:
grouping = groups[12]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final13 = cal_district_numbers(labels_check)
final13

YEAR OF ENTRY

Estimate!!YEAR OF ENTRY!!Population born outside the United States
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Native
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Native!!Entered 2010 or later
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Native!!Entered before 2010
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Foreign born
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Foreign born!!Entered 2010 or later
Estimate!!YEAR OF ENTRY!!Population born outside the United States!!Foreign born!!Entered before 2010


,0,1,2,3,4,5,6,7,8,9,10
Population born outside the United States counts,73583.60,25781.34,23252.30,18990.76,22881.48,23760.42,24682.80,22963.69,33921.80,22425.97,20663.84
Population born outside the United States counts percent,0.24,0.08,0.07,0.06,0.07,0.08,0.08,0.07,0.11,0.07,0.07
Population born outside the United States moe,3103.64,1842.69,1664.26,1585.29,1536.29,1622.59,1941.79,1911.79,2205.31,1526.52,1657.64
Population born outside the United States moa percent,0.03,0.04,0.04,0.05,0.04,0.04,0.05,0.05,0.04,0.04,0.05
Native counts,18273.75,2360.53,4384.45,1513.61,2593.21,1177.37,5254.09,3015.58,4152.74,4579.12,4476.56
Native counts percent,0.35,0.05,0.08,0.03,0.05,0.02,0.10,0.06,0.08,0.09,0.09
Native moe,1372.74,528.07,657.19,345.13,458.87,323.84,828.37,460.01,533.68,682.61,700.86
Native moa percent,0.05,0.14,0.09,0.14,0.11,0.17,0.10,0.09,0.08,0.09,0.10
Native!!Entered 2010 or later counts,5306.38,458.95,1490.26,513.56,590.39,370.02,1696.37,865.95,649.56,1125.98,895.58
Native!!Entered 2010 or later counts percent,0.38,0.03,0.11,0.04,0.04,0.03,0.12,0.06,0.05,0.08,0.06


### WORLD REGION OF BIRTH OF FOREIGN BORN

In [21]:
grouping = groups[13]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final14 = cal_district_numbers(labels_check)
final14

WORLD REGION OF BIRTH OF FOREIGN BORN

Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Europe
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Asia
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Africa
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Oceania
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Latin America
Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Northern America


,0,1,2,3,4,5,6,7,8,9,10
"Foreign-born population, excluding population born at sea counts",55309.85,23420.81,18867.85,17477.15,20288.28,22583.05,19428.71,19948.11,29769.07,17846.86,16187.28
"Foreign-born population, excluding population born at sea counts percent",0.21,0.09,0.07,0.07,0.08,0.09,0.07,0.08,0.11,0.07,0.06
"Foreign-born population, excluding population born at sea moe",2726.58,1770.44,1514.67,1545.07,1466.02,1576.46,1723.53,1822.26,2141.51,1445.74,1484.54
"Foreign-born population, excluding population born at sea moa percent",0.03,0.05,0.05,0.05,0.04,0.04,0.05,0.06,0.04,0.05,0.06
Europe counts,3787.81,594.43,648.80,123.58,323.32,195.80,1021.97,728.88,1730.37,1529.26,1278.78
Europe counts percent,0.32,0.05,0.05,0.01,0.03,0.02,0.09,0.06,0.14,0.13,0.11
Europe moe,619.14,175.83,207.44,97.96,142.39,145.28,472.46,200.05,348.53,312.61,263.75
Europe moa percent,0.10,0.18,0.19,0.48,0.27,0.45,0.28,0.17,0.12,0.12,0.13
Asia counts,12160.54,971.92,1768.94,530.05,1140.52,189.15,3598.60,5185.01,14214.48,5590.21,3714.57
Asia counts percent,0.25,0.02,0.04,0.01,0.02,0.00,0.07,0.11,0.29,0.11,0.08


### LANGUAGE SPOKEN AT HOME

In [22]:
grouping = groups[14]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final15 = cal_district_numbers(labels_check)
final15

LANGUAGE SPOKEN AT HOME

Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!English only
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Language other than English
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Language other than English!!Speak English less than very well""
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Spanish
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Spanish!!Speak English less than very well""
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Other Indo-European languages
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Other Indo-European languages!!Speak English less than very well""
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Asian and Pacific Islander languages
Estimate!!LANGUAGE SPOKEN AT HOME!!Population 5 years and over!!Asian and Pacific Islander languages!!Speak

,0,1,2,3,4,5,6,7,8,9,10
Population 5 years and over counts,499784.14,117404.59,131746.80,131673.45,122848.99,115572.92,155386.36,137690.81,156055.34,146937.98,139159.62
Population 5 years and over counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.08,0.07,0.08,0.08,0.08
Population 5 years and over moe,8560.80,3538.69,4052.61,3754.47,3781.17,3987.52,4975.76,4130.19,3777.14,4213.05,3534.79
Population 5 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02
English only counts,352120.56,59215.37,80875.36,68492.32,59531.76,42197.21,92768.14,81376.96,96968.80,108126.15,102678.36
English only counts percent,0.31,0.05,0.07,0.06,0.05,0.04,0.08,0.07,0.08,0.09,0.09
English only moe,7494.39,2651.99,3186.65,2856.08,2966.58,2701.50,4002.78,3428.97,3120.59,3754.98,3214.80
English only moa percent,0.01,0.03,0.02,0.03,0.03,0.04,0.03,0.03,0.02,0.02,0.02
Language other than English counts,147663.58,58189.22,50871.44,63181.13,63317.23,73375.71,62618.22,56313.85,59086.54,38811.83,36481.26
Language other than English counts percent,0.21,0.08,0.07,0.09,0.09,0.10,0.09,0.08,0.08,0.05,0.05


### ANCESTRY

In [23]:
grouping = groups[15]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final16 = cal_district_numbers(labels_check)
final16

ANCESTRY

Estimate!!ANCESTRY!!Total population
Estimate!!ANCESTRY!!Total population!!American
Estimate!!ANCESTRY!!Total population!!Arab
Estimate!!ANCESTRY!!Total population!!Czech
Estimate!!ANCESTRY!!Total population!!Danish
Estimate!!ANCESTRY!!Total population!!Dutch
Estimate!!ANCESTRY!!Total population!!English
Estimate!!ANCESTRY!!Total population!!French (except Basque)
Estimate!!ANCESTRY!!Total population!!French Canadian
Estimate!!ANCESTRY!!Total population!!German
Estimate!!ANCESTRY!!Total population!!Greek
Estimate!!ANCESTRY!!Total population!!Hungarian
Estimate!!ANCESTRY!!Total population!!Irish
Estimate!!ANCESTRY!!Total population!!Italian
Estimate!!ANCESTRY!!Total population!!Lithuanian
Estimate!!ANCESTRY!!Total population!!Norwegian
Estimate!!ANCESTRY!!Total population!!Polish
Estimate!!ANCESTRY!!Total population!!Portuguese
Estimate!!ANCESTRY!!Total population!!Russian
Estimate!!ANCESTRY!!Total population!!Scotch-Irish
Estimate!!ANCESTRY!!Total population!!Scottish
Estimat

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
Total population counts,538323.84,125014.68,143060.08,140873.10,132566.94,125056.03,167877.44,147037.45,166422.74,155155.67,149134.04
Total population counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.08,0.07,0.08,0.08,0.07
Total population moe,9093.05,3793.28,4486.09,4040.89,4164.44,4441.76,5753.03,4444.57,4049.76,4723.08,3877.11
Total population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02
American counts,19134.57,2850.62,3985.37,4375.71,3998.03,2858.43,5385.26,4357.25,5113.13,7416.48,7186.14
...,...,...,...,...,...,...,...,...,...,...,...
Welsh moa percent,0.14,0.33,0.29,0.86,0.42,3.38,0.31,0.22,0.27,0.35,0.21
West Indian (excluding Hispanic origin groups) counts,2196.72,90.02,490.42,68.16,241.02,58.93,619.16,312.13,452.78,604.03,540.62
West Indian (excluding Hispanic origin groups) counts percent,0.39,0.02,0.09,0.01,0.04,0.01,0.11,0.06,0.08,0.11,0.10
West Indian (excluding Hispanic origin groups) moe,725.88,106.51,221.66,102.77,207.82,97.20,357.66,226.07,223.11,305.86,268.49


### COMPUTERS AND INTERNET USE

In [24]:
grouping = groups[16]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final17 = cal_district_numbers(labels_check)
final17

COMPUTERS AND INTERNET USE

Estimate!!COMPUTERS AND INTERNET USE!!Total households
Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer
Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription


,0,1,2,3,4,5,6,7,8,9,10
Total households counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Total households counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total households moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Total households moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
With a computer counts,172704.62,45399.55,47184.25,43067.65,38179.33,34009.28,55551.49,52119.36,67691.45,63605.33,57355.70
With a computer counts percent,0.26,0.07,0.07,0.06,0.06,0.05,0.08,0.08,0.10,0.09,0.08
With a computer moe,2518.09,1337.24,1376.05,1210.25,995.86,1195.03,1527.46,1398.74,1545.06,1486.83,1192.17
With a computer moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01
With a broadband Internet subscription counts,163524.60,39447.76,41173.71,36778.34,35265.87,29704.17,51902.44,47751.43,63211.74,60939.93,52295.01
With a broadband Internet subscription counts percent,0.26,0.06,0.07,0.06,0.06,0.05,0.08,0.08,0.10,0.10,0.08


In [25]:
with pd.ExcelWriter('sa2020_ccd_household.xlsx') as writer:
    final1.to_excel(writer, sheet_name=groups[0][:30])
    final2.to_excel(writer, sheet_name=groups[1][:30])
    final3.to_excel(writer, sheet_name=groups[2][:30])
    final4.to_excel(writer, sheet_name=groups[3][:30])
    final5.to_excel(writer, sheet_name=groups[4][:30])
    final6.to_excel(writer, sheet_name=groups[5][:30])
    final7.to_excel(writer, sheet_name=groups[6][:30])
    final8.to_excel(writer, sheet_name=groups[7][:30])
    final9.to_excel(writer, sheet_name=groups[7][:30])
    final10.to_excel(writer, sheet_name=groups[7][:30])
    final11.to_excel(writer, sheet_name=groups[7][:30])        
    final12.to_excel(writer, sheet_name=groups[7][:30])
    final13.to_excel(writer, sheet_name=groups[7][:30])
    final14.to_excel(writer, sheet_name=groups[7][:30])
    final15.to_excel(writer, sheet_name=groups[7][:30])
    final16.to_excel(writer, sheet_name=groups[7][:30])
    final17.to_excel(writer, sheet_name=groups[7][:30])

In [26]:
len(groups)

17